In [1]:
import sympy as sym
import math as m
import numpy as np
import scipy.optimize
from sympy import pprint
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.optimize import minimize

import SIC_POVM_Jun27 as sic


Solution to the system: [ 2.02257517e-02  9.57524656e-03  2.63918877e-02 -3.32780738e-02
 -7.16435583e-02  5.59177884e-02  2.33328959e-02  2.02080964e-02
  9.55563634e-03  2.63742186e-02 -3.32911117e-02 -7.16670503e-02
  5.59155730e-02  2.30251418e-02  8.45344803e-02 -5.91283448e-02
 -7.49432695e-04 -1.91186105e-01  1.70204432e-01 -1.23321702e-03
  1.81471286e-04 -1.76818886e-02 -6.76455306e-03 -2.38586357e-02
  3.51536222e-02  7.50483159e-02 -5.56595614e-02  2.27915621e-02
 -1.21975092e-01 -1.56708022e-01 -3.35174957e-01  3.58968698e-02
  8.33105382e-01  1.51219913e+00 -3.02358070e-01]
Residuals: [-0.002467037918005472, 0.018725233725877897, 0.020955098411397956, 0.054998307218065826, -0.036771821914410356, 0.035830084536730505, 0.00013415280315851735, -0.002464884415455626, 0.018710651540159058, 0.020923755143631796, 0.05494290467463491, -0.03685936560027518, 0.03573526997985097, 0.0001335249328837506, -0.010311101003328457, 0.09380036626301669, 0.025657326603638664, 0.01779372009875

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:239: ComplexWarning: Casting complex values to real discards the imaginary part
  retval = _minpack._hybrd(func, x0, args, 1, xtol, maxfev,
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


In [17]:
print(sic.solution)
print(sic.residuals)
print(sic.residuals_sum)

[ 5.39947499e-02 -5.39947499e-02  3.41479678e-01 -3.41479678e-01
 -3.49382845e-01  3.49382845e-01  1.36024980e-01  5.43783422e-02
 -5.43783422e-02  3.37911719e-01 -3.37911719e-01 -3.59205510e-01
  3.59205510e-01  1.45687225e-01  7.02075490e-01 -7.02075490e-01
  2.27088003e-01 -2.27088003e-01 -4.98689480e-01  4.98689480e-01
  6.68160881e-01 -3.48936782e-02  3.48936782e-02 -4.66517243e-01
  4.66517243e-01  1.45544506e-02 -1.45544506e-02 -1.17615790e-01
  2.65383007e-13  1.00000000e+00 -8.95312658e-13  1.00000000e+00
  3.72621111e-13  1.00000000e+00  2.02656113e-11]
[1.4329289114494227e-14, 8.89843754237063e-14, -2.3941606956446403e-13, -1.4241885448740277e-11, -3.5830365414834474e-12, -5.655476087440547e-13, -3.3992242188665557e-12, 1.443108797144431e-14, 9.413303470040546e-14, -2.312395045494266e-13, -1.4496293054833131e-11, -3.953847428263887e-12, -6.227240945122503e-13, -3.41774148409909e-12, 1.8631890489990362e-13, -4.3787196091216174e-13, -9.35950937187077e-13, -1.596534016101714e-1